In [30]:
import numpy as np
import torch, torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import time

In [11]:
# 5 examples with 2 dimensional representation
random_input = Variable(torch.FloatTensor(5, 2, 1).normal_(), requires_grad=False)

In [12]:
random_input

tensor([[[-1.1172],
         [-3.2629]],

        [[ 0.9789],
         [ 2.3778]],

        [[-0.9737],
         [-0.4792]],

        [[ 0.1305],
         [ 1.0045]],

        [[-1.1650],
         [ 0.7274]]])

In [3]:
bi_grus = torch.nn.GRU(input_size=2, hidden_size=1, num_layers=1, batch_first=False, bidirectional=True)

In [4]:
reverse_gru = torch.nn.GRU(input_size=1, hidden_size=1, num_layers=1, batch_first=False, bidirectional=False)

In [5]:
reverse_gru.weight_ih_l0 = bi_grus.weight_ih_l0_reverse
reverse_gru.weight_hh_l0 = bi_grus.weight_hh_l0_reverse
reverse_gru.bias_ih_l0 = bi_grus.bias_ih_l0_reverse
reverse_gru.bias_hh_l0 = bi_grus.bias_hh_l0_reverse

In [6]:
bi_output, bi_hidden = bi_grus(random_input)

In [7]:
reverse_output, reverse_hidden = reverse_gru(random_input[np.arange(4, -1, -1), :, :])

In [31]:
num_epochs = 5
learning_rate = 0.001
embedding_dim = 20
hidden_dim = 10
model_path = "dummy-model"

In [32]:
class Model(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(Model, self).__init__()
        self.bi_grus = torch.nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=1, batch_first=False, bidirectional=True)
#         (Side note) The output shape of GRU in PyTorch when batch_firstis false:
#         output (seq_len, batch, hidden_size * num_directions)
#         h_n (num_layers * num_directions, batch, hidden_size)
        self.linear = nn.Linear(in_features=hidden_dim*2, out_features=2, bias=True)
        
    def forward(self, inputs):
        out_gru = self.bi_grus(inputs).view(1,-1) # 
        # forward_out = out_gru[:, :, :hidden_dim]
        # backward_out = out_gru[:, :, hidden_dim:]
        print("gru:", out_gru.shape)
        out_linear = self.linear(out_gru) #
        print("output: ", out_linear.shape)
        return out_linear

In [33]:
model = Model(embedding_dim=embedding_dim, hidden_dim=hidden_dim)

In [34]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

bi_grus.weight_ih_l0
bi_grus.weight_hh_l0
bi_grus.bias_ih_l0
bi_grus.bias_hh_l0
bi_grus.weight_ih_l0_reverse
bi_grus.weight_hh_l0_reverse
bi_grus.bias_ih_l0_reverse
bi_grus.bias_hh_l0_reverse
linear.weight
linear.bias


In [ ]:
simplemodel.train()
total_loss = 0.
start_time = time.time()

for epoch in range(1,num_epochs+1):
    print("- - - - - - - - - - - - - - - - - - - -")
    for j,(inputs, targets) in enumerate(data):
        inputs = torch.LongTensor(inputs)
        targets = torch.LongTensor(targets)
        output = model(inputs=inputs)
        logits = nn.Softmax()(output)
        loss = nn.BCELoss(weight=None, size_average=None, reduction='mean')(input=logits, target=y)
        optimizer.zero_grad()
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        curr_loss+=loss.item()

        elapsed = time.time() - start_time
        print('- | epoch {:3d} | lr {:02.3f} | '
                'loss {:5.2f}  - '.format(
            epoch, curr_loss))
        start_time = time.time()
    print("\nLoss :", curr_loss/j, "  ; Time: ",elapsed,"\n")

In [ ]:
torch.save(model, model_path)